# Image processing with satellite data

## Medium-scale computations on a virtual machine in GCP

This notebook performs calculations with a GeoTIFF dataset using XArray and Dask. We load and rescale Landsat 8 images and compute the normalized difference vegetation index (NDVI), which distinguishes green vegetation from areas of bare land or water.

We'll use 10 images of the Denver, USA area taken from May-September 2018.

![RGB image](https://landsat-pds.s3.amazonaws.com/c1/L8/033/033/LC08_L1TP_033033_20180706_20180717_01_T1/LC08_L1TP_033033_20180706_20180717_01_T1_thumb_small.jpg)

## Step 1: Import packages

In [ ]:
import dask
import json
import os
import rasterio
import requests
import rioxarray
import matplotlib.pyplot as plt
%matplotlib inline

## Step 2: Define input data/images

We are using 10 images from the [Landsat dataset on GCP](https://cloud.google.com/storage/docs/public-datasets/landsat) and each band is available as a separate GeoTIFF file.

In [ ]:
images = [
"LC08_L1TP_033033_20180417_20180501_01_T1",
"LC08_L1TP_033033_20180503_20180516_01_T1",
"LC08_L1TP_033033_20180519_20180605_01_T1",
"LC08_L1TP_033033_20180604_20180615_01_T1",
"LC08_L1TP_033033_20180706_20180717_01_T1",
"LC08_L1TP_033033_20180722_20180731_01_T1",
"LC08_L1TP_033033_20180807_20180815_01_T1",
"LC08_L1TP_033033_20180823_20180829_01_T1",
"LC08_L1TP_033033_20180908_20180912_01_T1",
"LC08_L1TP_033033_20180924_20180929_01_T1",
]

urls = []
for i in images:
    urls.append(["https://storage.googleapis.com/gcp-public-data-landsat/LC08/01/033/033/{}/{}_B5.TIF".format(i, i),  # nir
                 "https://storage.googleapis.com/gcp-public-data-landsat/LC08/01/033/033/{}/{}_B4.TIF".format(i, i),  # red
                 "https://storage.googleapis.com/gcp-public-data-landsat/LC08/01/033/033/{}/{}_MTL.txt".format(i, i)])  # mtl

## Step 3: Create XArray datasets

In [ ]:
red = []
nir = []
for i in urls:
    import xarray as xr
    red.append(rioxarray.open_rasterio(i[1], chunks={'band': 1, 'x': 1024, 'y': 1024}))
    nir.append(rioxarray.open_rasterio(i[0], chunks={'band': 1, 'x': 1024, 'y': 1024}))
    nir

## Step 4: Create a Dask cluster on the remote virtual machine

In [ ]:
import dask
from dask.distributed import Client, LocalCluster
cluster = LocalCluster(n_workers=16)
client = Client(cluster)
client

## Step 5: Rescale bands using Landsat metadata and Dask

The Landsat Level 1 images are delivered in a quantized format. This has to be [converted to top-of-atmosphere reflectance](https://landsat.usgs.gov/using-usgs-landsat-8-product) using the provided metadata. First we define convenience functions to load the rescaling factors and transform a dataset. The red band is band 4 and near infrared is band 5.

In [ ]:
def load_scale_factors(filename, band_number):
    metadata = {}
    response = requests.get(filename)
    data = response.text.splitlines()
    for line in data:
        name, var = line.partition("=")[::2]
        metadata[name.strip()] = var
    
    M_p = float(metadata["REFLECTANCE_MULT_BAND_{}".format(band_number)])
    A_p = float(metadata["REFLECTANCE_ADD_BAND_{}".format(band_number)])
    
    return M_p, A_p

def calculate_reflectance(ds, band_number, metafile):
    M_p, A_p = load_scale_factors(metafile, band_number)
    toa = M_p * ds + A_p
    return toa

red_toa = []
nir_toa = []
for i, j, k in zip(red, nir, urls):
    red_toa.append(calculate_reflectance(i, band_number=4, metafile=k[2]))
    nir_toa.append(calculate_reflectance(j, band_number=5, metafile=k[2]))

Because the transformation is composed of arithmetic operations, execution is delayed and the operations are parallelized automatically.

In [ ]:
print(red_toa[0].variable.data)

The resulting image has floating point data with magnitudes appropriate to reflectance. This can be checked by computing the range of values in an image using Dask:

In [ ]:
red_max, red_min, red_mean = dask.compute(
    red_toa[0].max(dim=['x', 'y']), 
    red_toa[0].min(dim=['x', 'y']),
    red_toa[0].mean(dim=['x', 'y'])
)
print(red_max.item())
print(red_min.item())
print(red_mean.item())

## Step 6: Calculate normalized difference vegetation index (NDVI) using Dask

Now that we have the image as reflectance values, we are ready to compute the NDVI using Dask.

$$
\text{NDVI} = \frac{\text{NIR} - \text{Red}}{\text{NIR} + \text{Red}}
$$

This highlights areas of healthy vegetation with high NDVI values, which appear as green in the image below.

In [ ]:
def compute_nvdi(nir_toa_single, red_toa_single, images):
    ndvi = (nir_toa_single - red_toa_single) / (nir_toa_single + red_toa_single)
    ndvi2d = ndvi.squeeze()

    fig = plt.figure(figsize=[12,12])
    im = ndvi2d.compute().plot.imshow(cmap='BrBG', vmin=-0.5, vmax=1)
    plt.axis('equal')
    plt.show()
    fig.savefig("output/" + images + ".png")

In [ ]:
from dask.distributed import progress, wait

output = client.map(compute_nvdi, nir_toa, red_toa, images)
progress(output)